This notebook creates a zarr dataset from all the files stored in the directory `'/storage/GOES/orthorectified/Fog2022_withtime/'`. The zarr dataset is stored at the path `'/storage/GOES/orthorectified/Fog2022_withtime_tmp.zarr'`. The saved zarr dataset is optimized for access along the time-index, i.e. it makes it very quick to grab time series for individual pixels. 

In [1]:
import os
import glob
import shutil
import xarray as xr
import zarr
from dask.distributed import Client, LocalCluster

In [2]:
fixed_image_folder = '/storage/GOES/orthorectified/Fog2022_withtime/'
zarr_output_path = '/storage/GOES/orthorectified/Fog2022_withtime.zarr'
tmp_zarr_output_path = '/storage/GOES/orthorectified/Fog2022_withtime_tmp.zarr'

In [ ]:
# Create a Dask cluster so we can watch the dask dashboard
workers = 6
ip_addres = 'http://j-lundquist-3.ce.washington.edu'
port=':8787'
threads = 2
cluster = LocalCluster(n_workers=workers, threads_per_worker=threads, dashboard_address=port)
client = Client(cluster)

In [12]:
# Grab all the netcdf files
nc_files = sorted(glob.glob(os.path.join(fixed_image_folder, '*.nc')))
nc_files

['/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M4_G17_s20221481500220_e20221481505120_c20221481508078_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M4_G17_s20221481505215_e20221481510115_c20221481514230_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M4_G17_s20221482005195_e20221482010095_c20221482012213_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M4_G17_s20221631515215_e20221631520115_c20221631522543_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M4_G17_s20221761510209_e20221761515109_c20221761518069_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M4_G17_s20221761515215_e20221761520115_c20221761525078_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M4_G17_s20221901510209_e20221901515109_c20221901517525_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M4_G17_s20221901515209_e20221901520109_c20221901526068

In [10]:
! ls -lah /storage/GOES/orthorectified/Fog2022_withtime/ | grep 2022148 | wc -l

289


In [ ]:
# Open all the files
ds = xr.open_mfdataset(nc_files, chunks={'time': 500})

In [ ]:
# Rechunk in to 1e8 byte sized chunks
ds['HT'].data = ds['HT'].data.rechunk(
    {0:'auto', 1:'auto', 2:'auto'},
    block_size_limit=1e8,
    balance=True
)

In [ ]:
# Create a temporary zarr file and write these chunks to disk
shutil.rmtree(tmp_zarr_output_path, ignore_errors=True)
ds.to_zarr(tmp_zarr_output_path)

In [ ]:
source_group = zarr.open(tmp_zarr_output_path)
source_array = source_group['HT']
print(source_group.tree())
print(source_array.info)
del source_group
del source_array

In [ ]:
# Rechunk along time dimension and write to disk
shutil.rmtree(zarr_output_path, ignore_errors=False)

In [ ]:
arr = ds['HT'].data.rechunk(
    {0:-1, 1:'auto', 2:'auto'}, 
    block_size_limit=1e8, 
    balance=True
)

t,y,x = arr.chunks[0][0], arr.chunks[1][0], arr.chunks[2][0]

In [ ]:
ds = xr.open_dataset(
    tmp_zarr_output_path,
    chunks={'time': t, 'latitude': y, 'longitude':x},
    engine='zarr'
)

ds['HT'].encoding = {'chunks': (t, y, x)}

ds.to_zarr(zarr_output_path)

In [ ]:
source_group = zarr.open(zarr_output_path)
source_array = source_group['HT']
print(source_group.tree())
print(source_array.info)
del source_group
del source_array